<a href="https://colab.research.google.com/github/bijuthottathil/dataengineer/blob/main/pyspark_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
!pip -q install findspark


In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages io.delta:delta-core_2.12:0.7.0 --conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog pyspark-shell'
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('delta_session').getOrCreate()

In [4]:
from delta.tables  import *

In [ ]:
df_final= df_json.withColumn("toppingexplode",explode("topping"))\
                .withColumn("topping_id",col("toppingexplode.id"))\
                .withColumn("topping_type",col("toppingexplode.type"))\
                .drop("topping","toppingexplode")\
                .withColumn("batterexplode",explode("batters.batter"))\
                .withColumn("batter_type",col("batterexplode.id"))\
                .withColumn("batter_type",col("batterexplode.type"))\
                .drop("batter","batterexplode")

df_final.show()

In [24]:
from re import MULTILINE
df=spark.read.format("json").load("/content/Bijucost.json",header=True,MULTILINE=True)

In [28]:
df_final= df.withColumn("valueexplode",explode("value"))\
                .withColumn("kind",col("valueexplode.kind"))\
                .withColumn("id",col("valueexplode.id"))\
                .withColumn("Name",col("valueexplode.name"))
df_final.show()

+--------------------+--------------------+---------+--------------------+--------------------+
|               value|        valueexplode|     kind|                  id|                Name|
+--------------------+--------------------+---------+--------------------+--------------------+
|[[/subscriptions/...|[/subscriptions/2...|   modern|/subscriptions/22...|d95487ba-f4c9-ffc...|
|[[/subscriptions/...|[/subscriptions/2...|notmodern|/subscriptions/22...|1148ce7a-dcf9-ed0...|
|[[/subscriptions/...|[/subscriptions/2...|   modern|/subscriptions/22...|6457bb55-9e12-9e4...|
|[[/subscriptions/...|[/subscriptions/2...|   modern|/subscriptions/22...|8a0a86dc-d2ce-9df...|
|[[/subscriptions/...|[/subscriptions/2...|   modern|/subscriptions/22...|e3742e6d-c305-ac9...|
|[[/subscriptions/...|[/subscriptions/2...|   modern|/subscriptions/22...|c4361910-21b8-88d...|
|[[/subscriptions/...|[/subscriptions/2...|   modern|/subscriptions/22...|323c2c82-41e3-5d0...|
|[[/subscriptions/...|[/subscriptions/2.

In [ ]:
df.write.format("delta").partitionBy("HIREDATE").mode("overwrite").saveAsTable("empnewnew")

In [ ]:
from sys import set_coroutine_origin_tracking_depth
from IPython.utils.path import target_outdated
from delta.tables import *

deltaTablePeople = DeltaTable.forPath(spark, '/content/spark-warehouse/empnew')
deltaTablePeopleUpdates = DeltaTable.forPath(spark, '/content/spark-warehouse/empnewnew')

dfUpdates = deltaTablePeopleUpdates.toDF()

deltaTablePeople.alias('people') \
  merge into empnewnew as target_outdated
  using source_view as source
  on target.
